# Seoul Bike - Predict rented bike count by weather and derive business implications

Version: <b>0.9</b><br>
author: <b>Hendrik Lammers</b>

The two possible scenarios known from a foreign report are:
* 70 % Probability of +2 °C of average temperature ceteris paribus
* 30 % Probability of +3 percentage points in the average humidity ceteris paribus

### Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

### Exploring Data and Preprocessing

In [ ]:
# The dataset is not encoded in utf-8, therefore the encoding needs to be changed
df_bike = pd.read_csv("./SeoulBikeData.csv", encoding="iso-8859-1")

In [ ]:
df_bike

In [ ]:
# Rename the variables to make them more easy to handle
def rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename({
        "Rented Bike Count":"Rented_Bikes",
        "Temperature(°C)":"Temperature",
        "Humidity(%)":"Humidity",
        "Wind speed (m/s)":"Wind_Speed",
        "Visibility (10m)":"Visibility",
        "Dew point temperature(°C)":"Dew_Point",
        "Solar Radiation (MJ/m2)":"Solar_Radiation",
        "Rainfall(mm)":"Rainfall",
        "Snowfall (cm)":"Snowfall",
        "Functioning Day":"Functioning_Day"}
        ,axis=1)
    return df

In [ ]:
df_bike = rename_columns(df_bike)

In [ ]:
def onehot(df: pd.DataFrame, drop_seasons: bool = True) -> pd.DataFrame:
    list_seasons = list(df["Seasons"].drop_duplicates())
    # Replace Seasons with one-hot encoding
    df[list_seasons[1:]] = pd.get_dummies(df_bike["Seasons"],drop_first=True)
    
    # Turn the string field 'Holiday' into an int field with 1 for Holiday and 0 for No Holiday.
    df["Holiday"] = df["Holiday"].map({"Holiday": 1, "No Holiday": 0})
    
    # Same for the field 'Functioning_Day'
    df["Functioning_Day"] = df["Functioning_Day"].map({"Yes": 1, "No": 0})
    
    if drop_seasons:
        # The legacy Seasons variable is then no longer needed.
        df = df.drop("Seasons",axis=1)
    return df

In [ ]:
df_bike = onehot(df_bike)

The field named "Functioning_Day" could cause confusion later on. Inspect these data.

In [ ]:
df_bike[df_bike["Functioning_Day"] == 0]

Obviously, the data set contains records where the bike rent wasn't working. Keeping these would imply disorder of prediction of Rented Bike Count. Therefore, they need to be disregarded.

(Another option would be to guess the values for these, but in this case there isn't enough information to do that)

#### Delete rows where the bike rent was not working

In [ ]:
def delete_defects(df: pd.DataFrame, defect_column: str = "Functioning_Day") -> pd.DataFrame:
    df = df[df[defect_column] == 1]
    df = df.drop(defect_column, axis=1)
    return df

In [ ]:
df_functioning = delete_defects(df_bike)

In [ ]:
# Check for inanppropriate data types
df_functioning.info()

In [ ]:
# Check for missing values
df_functioning.isnull().any()

In [ ]:
# Get standard measures of location
df_functioning.describe()

In [ ]:
# Look at the distribution of the features and the target variable which is obviously right-skewed.
df_functioning.hist(figsize=(20,15), bins=20)
plt.show()

In [ ]:
# Lastly, Look at the final dataframe to see if there is anything out of the ordinary
df_functioning

#### Check for correlation of feature variables

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(df_functioning.corr(),cmap="coolwarm",linecolor="white",linewidths=1)
plt.plot()

Temperature and Humidity are correlated with Dewpoint - which is not surprising, as Dewpoints are dependent on Temperature and Humidity.<br>
Also, the correlation of temperature and is_summer and is_winter respectively can be observed.

### Model the data

In [ ]:
X = df_functioning.drop(["Date","Hour","Rented_Bikes"], axis=1)
y = df_functioning["Rented_Bikes"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# As we have not many observations, level the test-size down a bit from standard 0.25 to get more training data.
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

#### Try linear regression first

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

In [ ]:
lr_prediction = lr.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
r2_score(y_test, lr_prediction)

In [ ]:
mean_squared_error(y_test, lr_prediction, squared=False)

That was no good prediction obviously, the results are pretty poor. Try another model.

#### Try Gradient Boosting Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gb = GradientBoostingRegressor(n_estimators=500,random_state=1)

In [ ]:
gb.fit(X_train, y_train)

In [ ]:
gb_prediction = gb.predict(X_test)

In [ ]:
r2_score(y_test, gb_prediction)

In [ ]:
mean_squared_error(y_test, gb_prediction, squared=False)

Gradient Boosting Regressor has much better performance, but still there is a pretty high RMSE.

#### Try Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=500, random_state=1) # Values are arbitrarily chosen - 500 estimators is usually a good starting point.

In [ ]:
rf.fit(X_train, y_train)

In [ ]:
rf_prediction = rf.predict(X_test)

In [ ]:
r2_score(y_test, rf_prediction)

In [ ]:
mean_squared_error(y_test,rf_prediction,squared=False) # Calculate Root Mean Squared Error of RF Prediction

Random Forest has the best performance so far, though RMSE is still pretty high. Therefore choose Random Forest.

### Hyperparameter Optimization

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Try to optimize number of estimators and max_features. Also, try using the whole dataset for building each tree.
param_grid = [
    {"n_estimators": [10,100,300,500], "max_features": [9,10,11,12]},
    {"bootstrap": [False], "n_estimators": [10,100,300,500], "max_features": [9,10,11,12]}
]

In [ ]:
rf_2 = RandomForestRegressor()
grid_search = GridSearchCV(rf_2, param_grid, cv=5, scoring="neg_mean_squared_error",return_train_score=True)
grid_search.fit(X,y)

In [ ]:
grid_search.best_params_

9 is the lowest value tried out for max_features. Therefore try Grid Search again with lower values for max_features.

In [ ]:
param_grid_2 = [{"n_estimators": [80,100,120], "max_features":[1,2,3,4,5]}]

In [ ]:
rf_3 = RandomForestRegressor()
grid_search_2 = GridSearchCV(rf_3, param_grid_2, cv=5, scoring="neg_mean_squared_error", return_train_score=True)
grid_search_2.fit(X,y)

In [ ]:
grid_search_2.best_params_

2 seems to be the optimal value for max_features.

In [ ]:
rf_4 = RandomForestRegressor(max_features=2)
rf_4.fit(X_train,y_train)
rf4_pred = rf_4.predict(X_test)

In [ ]:
mean_squared_error(y_test,rf4_pred,squared=False)

In [ ]:
r2_score(y_test,rf4_pred)

Still a pretty high RMSE. Probably the observations are not very generalizable. A next step could be to take more observations to better generalize.<br>
Another possibility is a bias by missing out important features. The R²-Score is 71 %, meaning the features can explain around 71 % of the variance of the target variable. A second next step could therefore be to try to enrich the observations by more features to be able to explain more of the variance.

### Check for most important features

#### First Random Forest Regressor (no hyperparameter tuning)

In [ ]:
feature_importances = pd.DataFrame(rf.feature_importances_, columns=["feature importance"], index=X_train.columns).sort_values("feature importance", ascending=False)

In [ ]:
feature_importances

#### Random Forest Regressor with optimized hyperparameters

In [ ]:
feature_importances_opt = pd.DataFrame(rf_4.feature_importances_, columns=["feature importance"], index=X_train.columns).sort_values("feature importance", ascending=False)

In [ ]:
feature_importances_opt

### Predict Scenario 1: +2 °C in average temperature ceteris paribus

In [ ]:
X_sc1 = X.copy()

There are many permutations of temperature values that form an average of +2 °C against the observed temperatures.<br>
A reasonable assumption could be that every temperature value rises by 2 degrees. To get that, simply add +2 on each temperature value.

In [ ]:
X_sc1["Temperature"] = [temperature+2 for temperature in X_sc1["Temperature"]]

In [ ]:
rf_sc1_prediction = rf_4.predict(X_sc1)

In [ ]:
chg_sc1 = (rf_sc1_prediction.sum() - y.sum()) / y.sum()

In [ ]:
print("Predicted change in Bike Rentals in case of scenario 1: {0} %.".format(round(chg_sc1 * 100, 2)))

### Predict Scenario 2: +3 percentage points in average humidity ceteris paribus

In [ ]:
X_sc2 = X.copy()

In [ ]:
X_sc2["Humidity"] = [humidity+3 if humidity+3 < 100 else 100 for humidity in X_sc2["Humidity"]]

In [ ]:
X_sc2

In [ ]:
rf_sc2_prediction = rf_4.predict(X_sc2)

In [ ]:
chg_sc2 = (rf_sc2_prediction.sum() - y.sum()) / y.sum()

In [ ]:
print("Predicted change in Bike Rentals in case of scenario 1: {0} %.".format(round(chg_sc2 * 100, 2)))

In [ ]:
expct_value = 0.7 * chg_sc1 + 0.3 * chg_sc2

In [ ]:
print("The expected growth for Demand Change in Bike Rentals is, according to the probabilities of scenario 1 and scenario 2 respectively, {0} %.".format(round(expct_value * 100, 2)))